In [ ]:
#Criando os clusters
#Importando bibliotecas
%pip install kmodes
%pip install numpy
%pip install pandas
%pip install plotly

import pandas as pd
import numpy as np
from kmodes.kmodes import KModes
from kmodes.util.dissim import jaccard_dissim_binary
import plotly.graph_objects as go

In [4]:
#Carregando os dados
dataset = pd.read_parquet("TMDB_movie_dataset_v11(com correçao)_updated.parquet")

In [ ]:
#Pareto paises
def diagrama_pareto(data_frame, coluna, titulo):

    #Contando as aparições de dados
    dados_separados = data_frame[coluna].str.split(', ').explode()
    aparicoes_dados = dados_separados.value_counts().sort_values(ascending=False)
    agrupado = aparicoes_dados.reset_index(name='Contagem')
    agrupado = agrupado.sort_values('Contagem', ascending=False)

    #Criando a série do somas acumulativas
    agrupado['Acumulado'] = agrupado['Contagem'].cumsum() / agrupado['Contagem'].sum() * 100

    #Criando a barra 'outros' após a linha atingir os 70%
    agrupado_principal = agrupado[agrupado['Acumulado'] <= 70]
    agrupado_outros = agrupado[agrupado['Acumulado'] > 70]
    outros_contagem = agrupado_outros['Contagem'].sum()
    linha_outros = pd.DataFrame({coluna: ['Outros'], 'Contagem': [outros_contagem], 'Acumulado': [100]})
    agrupado_principal = pd.concat([agrupado_principal, linha_outros], ignore_index=True)
    return agrupado_principal


paises_principais= diagrama_pareto(dataset, 'production_countries', 'Produção por País')

print(paises_principais)

In [ ]:
# Pareto linguas
def diagrama_pareto_linguas(data_frame, coluna, titulo):

    # Contando as aparições de dados
    dados_separados = data_frame[coluna].str.split(', ').explode()
    aparicoes_dados = dados_separados.value_counts().sort_values(ascending=False)
    agrupado = aparicoes_dados.reset_index(name='Contagem')
    agrupado = agrupado.sort_values('Contagem', ascending=False)

    # Criando a série do somas acumulativas
    agrupado['Acumulado'] = agrupado['Contagem'].cumsum() / agrupado['Contagem'].sum() * 100

    # Criando a barra 'outros' após a linha atingir os 70%
    agrupado_principal = agrupado[agrupado['Acumulado'] <= 70]
    agrupado_outros = agrupado[agrupado['Acumulado'] > 70]
    outros_contagem = agrupado_outros['Contagem'].sum()
    linha_outros = pd.DataFrame({coluna: ['Outros'], 'Contagem': [outros_contagem], 'Acumulado': [100]})
    agrupado_principal = pd.concat([agrupado_principal, linha_outros], ignore_index=True)
    return agrupado_principal

linguas_principais = diagrama_pareto_linguas(dataset, 'spoken_languages', 'Línguas Faladas')

print(linguas_principais)


In [ ]:
#onehotencoding paises
def onehotencoding(df, coluna):
    a_clusterizar = ['id', 'title', coluna]
    dataset_cluster = df[a_clusterizar].copy()
    
    # Fill None values with an empty string
    dataset_cluster[coluna] = dataset_cluster[coluna].fillna('')
    
    # Obtendo os valores únicos da coluna
    valores = paises_principais['production_countries'].values
    
    # Criando um DataFrame com todas as colunas de uma vez para evitar erro de defragmentação
    one_hot_data = {
        valor: dataset_cluster[coluna].apply(lambda x: 1 if valor in x else 0)
        for valor in valores
    }
    dataset_coluna = pd.DataFrame(one_hot_data, index=dataset_cluster.index)
    
    return dataset_coluna

onehotpaises = onehotencoding(dataset, 'production_countries')

print(onehotpaises.columns)

In [ ]:
#Teste para checar se a tabela binária que passará pela clusterização foi feita corretamente
onehotpaises.head()

In [ ]:
# onehotencoding linguas
def onehotencoding_linguas(df, coluna):
    a_clusterizar = ['id', 'title', coluna]
    dataset_cluster = df[a_clusterizar].copy()
    
    # Fill None values with an empty string
    dataset_cluster[coluna] = dataset_cluster[coluna].fillna('')
    
    # Obtendo os valores únicos da coluna
    valores = linguas_principais['spoken_languages'].values
    
    # Criando um DataFrame com todas as colunas de uma vez para evitar erro de defragmentação
    one_hot_data = {
        valor: dataset_cluster[coluna].apply(lambda x: 1 if valor in x else 0)
        for valor in valores
    }
    dataset_coluna = pd.DataFrame(one_hot_data, index=dataset_cluster.index)
    
    return dataset_coluna

onehotlinguas = onehotencoding_linguas(dataset, 'spoken_languages')

print(onehotlinguas.columns)

In [ ]:
#Teste para checar se a tabela binária que passará pela clusterização foi feita corretamente
onehotlinguas.head()

In [12]:
#onehotencoding
def onehotencoding(df, coluna):
    a_clusterizar = ['id', 'title', coluna]
    dataset_cluster = df[a_clusterizar].copy()
    
    # Obtendo os valores únicos da coluna
    valores = dataset_cluster[coluna].str.split(', ').explode().unique()
    
    # Criando um DataFrame com todas as colunas de uma vez para evitar erro de defragmentação
    one_hot_data = {
        valor: dataset_cluster[coluna].apply(lambda x: 1 if valor in x else 0)
        for valor in valores
    }
    dataset_coluna = pd.DataFrame(one_hot_data, index=dataset_cluster.index)
    
    return dataset_coluna

In [14]:
#Clusterização
#Criando a tabela utilizando one-hot encoding
#Criando a tabela binária com os gêneros

# Fill None values with an empty string
dataset['genres'] = dataset['genres'].fillna('')

bin_generos = onehotencoding(dataset, 'genres')

In [ ]:
#Teste para checar se a tabela binária que passará pela clusterização foi feita corretamente
bin_generos.head()

In [ ]:
# Juntando os atributos spoken_languages, production_countries e genres
combined_attributes = pd.concat([onehotlinguas, onehotpaises, bin_generos], axis=1)

# Exibindo as primeiras linhas do dataframe combinado
print(combined_attributes.head())

In [16]:
#elbowmethod
def elbowmethod(tabela_binaria):
    #Lista para preencher com a série de wcss
    wcss = []

    #Preenchendo a lista
    for k in range(1, 11):
        km = KModes(n_clusters=k, init='random', n_init=10, verbose=1, max_iter = 100, cat_dissim = jaccard_dissim_binary, random_state = 42)
        km.fit(tabela_binaria)
        wcss.append(km.cost_)
        
    #Criando o gráfico com a Plotly para demonstrar o resultado do Elbow Method
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=list(range(1, 11)),
        y=wcss,
        mode='lines+markers',
        name='WCSS',
        line=dict(color='royalblue', dash='dash'),
        marker=dict(size=8)
    ))
    
    fig.update_layout(
        title='Elbow Method',
        xaxis_title='Número de Clusters (k)',
        yaxis_title='WCSS',
        xaxis=dict(tickmode='linear', tick0=1, dtick=1),
        template='plotly_white'
    )
    
    return fig

In [ ]:
#Aplicando o elbowmethod
serie_wcss = elbowmethod(bin_generos)
serie_wcss.show()

In [ ]:
#Criando os clusters com o valor ideal de k
clusters_generos = KModes(n_clusters=6, init='random', n_init=10, verbose=2, max_iter = 100, cat_dissim = jaccard_dissim_binary, random_state = 42)
clusters_generos.fit(bin_generos)

In [17]:
#Adicionando a coluna 'cluster'
dataset['cluster'] = clusters_generos.labels_

In [ ]:
#Exibindo o dataframe agora com uma coluna 'cluster' apresentando o cluster em que cada registro ficou
dataset.head(10)

In [ ]:
# Lista dos países principais
selected_countries = ['United States of America', 'France', 'Germany', 'Japan', 'United Kingdom', 'Canada', 'India', 'Italy', 'Brazil', 'Spain', 'Mexico', 'China']

# Função para criar as colunas dos países principais e a coluna 'Outros'
def create_country_columns(df, countries):
    # Fill None values with an empty string
    df['production_countries'] = df['production_countries'].fillna('')
    for country in countries:
        df[country] = df['production_countries'].apply(lambda x: 1 if country in x else 0)
    df['Outros'] = df['production_countries'].apply(lambda x: 1 if not any(country in x for country in countries) else 0)
    return df

# Aplicando a função ao dataset
dataset = create_country_columns(dataset, selected_countries)

# Exibindo as primeiras linhas do dataframe atualizado
dataset.head()

In [ ]:
from kmodes.kmodes import KModes
from kmodes.util.dissim import jaccard_dissim_binary

# Instalar o kmodes
%pip install kmodes

# Criar a tabela binária para os principais países
bin_paises = onehotpaises

# Remover linhas com todos os valores zero
bin_paises = bin_paises[(bin_paises.T != 0).any()]

# Realizar a clusterização usando o KModes
clusters_paises = KModes(n_clusters=6, init='random', n_init=10, verbose=2, max_iter=100, cat_dissim=jaccard_dissim_binary, random_state=42)
clusters_paises.fit(bin_paises)

# Adicionar a coluna 'cluster' ao dataset
dataset['cluster_paises'] = clusters_paises.labels_

# Exibir as primeiras linhas do dataframe atualizado
dataset.head()

In [ ]:
from kmodes.kmodes import KModes
from kmodes.util.dissim import jaccard_dissim_binary

# Criar a tabela binária para os gêneros
bin_generos = onehotencoding(dataset, 'genres')

# Remover linhas com todos os valores zero
bin_generos = bin_generos[(bin_generos.T != 0).any()]

# Realizar a clusterização usando o KModes
clusters_generos = KModes(n_clusters=6, init='random', n_init=10, verbose=2, max_iter=100, cat_dissim=jaccard_dissim_binary, random_state=42)
clusters_generos.fit(bin_generos)

# Adicionar a coluna 'cluster_generos' ao dataset
dataset['cluster_generos'] = clusters_generos.labels_

# Exibir as primeiras linhas do dataframe atualizado
dataset.head()


In [ ]:
import plotly.express as px

# Adicionando a coluna 'cluster_paises' ao dataset
dataset['cluster_paises'] = clusters_paises.labels_

# Plotando o gráfico de dispersão
fig = px.scatter(
    dataset,
    x='id',
    y='cluster_paises',
    color='cluster_paises',
    title='Gráfico de Dispersão dos Clusters de Países',
    labels={'id': 'ID do Filme', 'cluster_paises': 'Cluster'}
)

fig.show()

In [ ]:
import plotly.express as px

# Adicionando a coluna 'cluster_generos' ao dataset
dataset['cluster_generos'] = clusters_generos.labels_

# Plotando o gráfico de dispersão
fig = px.scatter(
    dataset,
    x='id',
    y='cluster_generos',
    color='cluster_generos',
    title='Gráfico de Dispersão dos Clusters de Gêneros',
    labels={'id': 'ID do Filme', 'cluster_generos': 'Cluster'}
)

fig.show()